In [17]:
import segmentation_models_pytorch as smp
import torch
import torchvision
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, dataloader
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [6]:
device = torch.device("cuda:0")

In [7]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [8]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [9]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [12]:
for x in tqdm(csv_file['fixed_inputs'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 443625.25it/s]


In [13]:
for x in tqdm(csv_file['mask_paths'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 430133.93it/s]


In [14]:
csv_file['fixed_inputs'].values.tolist()[0]

'../../files/train_v2/00003e153.jpg'

In [15]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [18]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [19]:
train, val, test = split_datasets(csv_file)

In [23]:
class Version1Dataset(Dataset):
    def __init__(self, csv_file):
        self.input_images = csv_file['fixed_inputs'].values
        self.mask_images = csv_file['mask_paths'].values
    
    def __len__(self):
        return len(self.input_images)
    
    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.input_images[idx])
        img = torchvision.io.decode_jpeg(img, torchvision.ImageReadMode.RGB)
        mask = torchvision.io.read_file(self.mask_images[idx])

In [24]:
Version1Dataset(train)

['../../files/train_v2/292058d5d.jpg' '../../files/train_v2/39e58e449.jpg'
 '../../files/train_v2/e5f4def1e.jpg' ...
 '../../files/train_v2/9b53aa9e1.jpg' '../../files/train_v2/4153eda7d.jpg'
 '../../files/train_v2/45d940896.jpg']


In [5]:
model = smp.Unet(
    encoder_name="inceptionv4",
    encoder_weights=None,
    in_channels=3,             
    classes=1,                
)

In [7]:
model = model.to(device)

In [10]:
x = torch.ones([1, 3, 512, 512]).to(device)

In [12]:
pred = model(x)